In [1]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts  instrument.partitionByInstrument(midi)
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch)+"^"+str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder)+"^"+str(element.quarterLength))
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 30

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()


Parsing midi_songs\C_0fithos.mid
Parsing midi_songs\C_8.mid
Parsing midi_songs\C_ahead_on_our_way_piano.mid
Parsing midi_songs\C_AT.mid
Parsing midi_songs\C_balamb.mid
Parsing midi_songs\C_bcm.mid
Parsing midi_songs\C_BlueStone_LastDungeon.mid
Parsing midi_songs\C_braska.mid
Parsing midi_songs\C_caitsith.mid
Parsing midi_songs\C_Cids.mid
Parsing midi_songs\C_cosmo.mid
Parsing midi_songs\C_costadsol.mid
Parsing midi_songs\C_dayafter.mid
Parsing midi_songs\C_decisive.mid
Parsing midi_songs\C_dontbeafraid.mid
Parsing midi_songs\C_DOS.mid
Parsing midi_songs\C_electric_de_chocobo.mid
Parsing midi_songs\C_Eternal_Harvest.mid
Parsing midi_songs\C_EyesOnMePiano.mid
Parsing midi_songs\C_ff11_awakening_piano.mid
Parsing midi_songs\C_ff1battp.mid
Parsing midi_songs\C_FF3_Battle_(Piano).mid
Parsing midi_songs\C_FF3_Third_Phase_Final_(Piano).mid
Parsing midi_songs\C_ff4-airship.mid
Parsing midi_songs\C_Ff4-BattleLust.mid
Parsing midi_songs\C_ff4-fight1.mid
Parsing midi_songs\C_ff4-town.mid
Parsing 

KeyboardInterrupt: 

In [ ]:
import os
from music21 import converter, pitch, interval

# Define save directory
save_dir = '../midi_songs/'

'''
Data collection
'''
# Identify list of MIDI files
songList = os.listdir(save_dir)

# Create empty list for scores
originalScores = []

# Load and make list of stream objects
for song in songList:
    score = converter.parse(save_dir+song)
    originalScores.append(score)

In [ ]:
from music21 import instrument

# Define function to test whether stream is monotonic
def monophonic(stream):
    try:
        length = len(instrument.partitionByInstrument(stream).parts)
    except:
        length = 0
    return length == 1

# Merge notes into chords
originalScores = [song.chordify() for song in originalScores]

In [ ]:
from music21 import note, chord

originalChords = [[] for _ in originalScores]
originalDurations = [[] for _ in originalScores]
originalKeys = []

# Extract notes, chords, durations, and keys
for i, song in enumerate(originalScores):
    originalKeys.append(str(song.analyze('key')))
    for element in song:
        if isinstance(element, note.Note):
            originalChords[i].append(element.pitch)
            originalDurations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            originalChords[i].append('.'.join(str(n) for n in element.pitches))
            originalDurations[i].append(element.duration.quarterLength)
    print(str(i))

In [ ]:
print(originalKeys)

In [ ]:
import glob
import os
import music21

#converting everything into the key of C major or A minor

# major conversions
majors = dict([("A-", 4),("A", 3),("B-", 2),("B", 1),("C", 0),("D-", -1),("D", -2),("E-", -3),("E", -4),("F", -5),("G-", 6),("G", 5),("G#", 4),("A", 3),("A#", 2),("B", 1),("C", 0),("C#", -1),("D", -2),("D#", -3),("E", -4),("F", -5),("F#", 6),("G", 5)])
minors = dict([("A-", 1),("A", 0),("B-", -1),("B", -2),("C", -3),("D-", -4),("D", -5),("E-", 6),("E", 5),("F", 4),("G-", 3),("G", 2),("G#", 4),("A", 3),("A#", 2),("B", 1),("C", 0),("C#", -1),("D", -2),("D#", -3),("E", -4),("F", -5),("F#", 6),("G", 5)])

songList = os.listdir(save_dir)

for file in songList:
    score = music21.converter.parse(save_dir+file)
    key = score.analyze('key')
#    print key.tonic.name, key.mode
    if key.mode == "major":
        halfSteps = majors[key.tonic.name]
        
    elif key.mode == "minor":
        halfSteps = minors[key.tonic.name]
    
    newscore = score.transpose(halfSteps)
    key = newscore.analyze('key')
    print(key.tonic.name, key.mode)
    newFileName = "C_" + file
    newscore.write('midi',newFileName)

C minor


'C_0fithos.mid'

C major


'C_8.mid'

C major


...

C major


'C_braska.mid'

C major


'C_caitsith.mid'

C major
